### tfhw04
继续优化手写数字分类程序，把Testing Accuracy提升到98%以上

试着加了一层100个神经元的隐层  
* 如果隐层和输出层的激活函数都使用tanh,外加一层softmax，精度到91%左右  
* 如果隐层和输出层的激活函数都使用relu,外加一层softmax，精度到85%左右  
* 如果隐层和输出层的激活函数都使用relu,不使用softmax，精度到97%左右  

为了进一步提升准确率，有以下几种思路：  
* 使用交叉熵  
但是使用交叉熵的目的是快速收敛，并不是提高准确率。  
这里只是试一下，softmax+softmax的交叉熵。  
tanh+softmax，使用均方差作为损失函数，训练41个epoch，准确率0.9245  
tanh+softmax，使用tf.nn.softmax_cross_entropy_with_logits作为损失函数，训练41个epoch，准确率0.9512  
tanh+softmax，使用tf新版本tf.nn.softmax_cross_entropy_with_logits_v2作为损失函数，训练41个epoch，准确率0.9521  
* 使用别的优化函数 
这里主要看到学习率对于最后结果的影响。
```
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)  #学习率为0.2时准确率0.2082；学习率为0.01时准确率0.9599;0.001时0.9699
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)  #学习率为0.2时准确率0.2672；学习率为0.01时准确率0.9674；0.001时0.9755
```


In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
import tensorflow as tf
from utils.dl.tf_wrapper import tf_wrapper as tfw
from tensorflow.examples.tutorials.mnist import input_data

/home/ian/installed/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

In [12]:
#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
#创建一个简单的神经网络
W = tf.Variable(tf.random_normal([784,100],0,0.02))
b = tf.Variable(tf.zeros([100]))
L1 = tf.nn.relu(tf.matmul(x,W)+b)
#创建一个简单的神经网络
W2 = tf.Variable(tf.random_normal([100,10],0,0.02))
b2 = tf.Variable(tf.zeros([10]))
prediction = tf.nn.relu(tf.matmul(L1,W2)+b2)
# prediction = tf.nn.softmax(L2)
#二次代价函数
loss = tf.reduce_mean(tf.square(y-prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [13]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(41):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Iter 0,Testing Accuracy 0.8181
Iter 1,Testing Accuracy 0.8589
Iter 2,Testing Accuracy 0.8812
Iter 3,Testing Accuracy 0.8909
Iter 4,Testing Accuracy 0.9001
Iter 5,Testing Accuracy 0.9071
Iter 6,Testing Accuracy 0.9419
Iter 7,Testing Accuracy 0.9465
Iter 8,Testing Accuracy 0.9489
Iter 9,Testing Accuracy 0.9522
Iter 10,Testing Accuracy 0.9541
Iter 11,Testing Accuracy 0.9556
Iter 12,Testing Accuracy 0.9563
Iter 13,Testing Accuracy 0.9587
Iter 14,Testing Accuracy 0.9593
Iter 15,Testing Accuracy 0.9608
Iter 16,Testing Accuracy 0.9627
Iter 17,Testing Accuracy 0.9633
Iter 18,Testing Accuracy 0.9637
Iter 19,Testing Accuracy 0.9646
Iter 20,Testing Accuracy 0.9658
Iter 21,Testing Accuracy 0.9672
Iter 22,Testing Accuracy 0.9662
Iter 23,Testing Accuracy 0.9685
Iter 24,Testing Accuracy 0.9692
Iter 25,Testing Accuracy 0.9689
Iter 26,Testing Accuracy 0.97
Iter 27,Testing Accuracy 0.97
Iter 28,Testing Accuracy 0.9697
Iter 29,Testing Accuracy 0.9706
Iter 30,Testing Accuracy 0.9713
Iter 31,Testing Accura

### 使用交叉熵

In [10]:
#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
#创建一个简单的神经网络
W = tf.Variable(tf.random_normal([784,100],0,0.02))
b = tf.Variable(tf.zeros([100]))
L1 = tf.nn.tanh(tf.matmul(x,W)+b)
#创建一个简单的神经网络
W2 = tf.Variable(tf.random_normal([100,10],0,0.02))
b2 = tf.Variable(tf.zeros([10]))
L2 = tf.nn.tanh(tf.matmul(L1,W2)+b2)
prediction = tf.nn.softmax(L2)
#二次代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(41):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Iter 0,Testing Accuracy 0.8318
Iter 1,Testing Accuracy 0.8887
Iter 2,Testing Accuracy 0.9024
Iter 3,Testing Accuracy 0.9098
Iter 4,Testing Accuracy 0.9133
Iter 5,Testing Accuracy 0.9182
Iter 6,Testing Accuracy 0.9192
Iter 7,Testing Accuracy 0.9228
Iter 8,Testing Accuracy 0.9255
Iter 9,Testing Accuracy 0.9264
Iter 10,Testing Accuracy 0.9283
Iter 11,Testing Accuracy 0.9302
Iter 12,Testing Accuracy 0.9312
Iter 13,Testing Accuracy 0.934
Iter 14,Testing Accuracy 0.9343
Iter 15,Testing Accuracy 0.9347
Iter 16,Testing Accuracy 0.9366
Iter 17,Testing Accuracy 0.9367
Iter 18,Testing Accuracy 0.9386
Iter 19,Testing Accuracy 0.9401
Iter 20,Testing Accuracy 0.9396
Iter 21,Testing Accuracy 0.941
Iter 22,Testing Accuracy 0.9423
Iter 23,Testing Accuracy 0.9432
Iter 24,Testing Accuracy 0.9443
Iter 25,Testing Accuracy 0.9454
Iter 26,Testing Accuracy 0.9453
Iter 27,Testing Accuracy 0.9458
Iter 28,Testing Accuracy 0.9468
Iter 29,Testing Accuracy 0.9478
Iter 30,Testing Accuracy 0.948
Iter 31,Testing Accur

In [19]:
#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
#创建一个简单的神经网络
W = tf.Variable(tf.random_normal([784,100],0,0.02))
b = tf.Variable(tf.zeros([100]))
L1 = tf.nn.tanh(tf.matmul(x,W)+b)
#创建一个简单的神经网络
W2 = tf.Variable(tf.random_normal([100,10],0,0.02))
b2 = tf.Variable(tf.zeros([10]))
L2 = tf.nn.tanh(tf.matmul(L1,W2)+b2)
prediction = tf.nn.softmax(L2)
#二次代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)  #学习率为0.2时准确率0.2082；学习率为0.01时准确率0.9599;0.001时0.9699
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)  #学习率为0.2时准确率0.2672；学习率为0.01时准确率0.9674；0.001时0.9755

#初始化变量
init = tf.global_variables_initializer()
tf.nn.dropout(L1,keep_prob)
#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(41):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Iter 0,Testing Accuracy 0.9169
Iter 1,Testing Accuracy 0.9356
Iter 2,Testing Accuracy 0.9468
Iter 3,Testing Accuracy 0.9523
Iter 4,Testing Accuracy 0.9572
Iter 5,Testing Accuracy 0.9587
Iter 6,Testing Accuracy 0.9617
Iter 7,Testing Accuracy 0.9638
Iter 8,Testing Accuracy 0.9654
Iter 9,Testing Accuracy 0.9667
Iter 10,Testing Accuracy 0.9679
Iter 11,Testing Accuracy 0.9686
Iter 12,Testing Accuracy 0.9693
Iter 13,Testing Accuracy 0.9703
Iter 14,Testing Accuracy 0.9698
Iter 15,Testing Accuracy 0.97
Iter 16,Testing Accuracy 0.9706
Iter 17,Testing Accuracy 0.9722
Iter 18,Testing Accuracy 0.9721
Iter 19,Testing Accuracy 0.9722
Iter 20,Testing Accuracy 0.972
Iter 21,Testing Accuracy 0.973
Iter 22,Testing Accuracy 0.9725
Iter 23,Testing Accuracy 0.9741
Iter 24,Testing Accuracy 0.9735
Iter 25,Testing Accuracy 0.9729
Iter 26,Testing Accuracy 0.975
Iter 27,Testing Accuracy 0.9735
Iter 28,Testing Accuracy 0.9736
Iter 29,Testing Accuracy 0.9741
Iter 30,Testing Accuracy 0.9755
Iter 31,Testing Accurac

### 增加一层隐层，神经元的个数300

In [20]:
#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
#创建一个简单的神经网络
W = tf.Variable(tf.random_normal([784,300],0,0.02))
b = tf.Variable(tf.zeros([300]))
L1 = tf.nn.tanh(tf.matmul(x,W)+b)
#创建一个简单的神经网络
W1 = tf.Variable(tf.random_normal([300,100],0,0.02))
b1 = tf.Variable(tf.zeros([100]))
L11 = tf.nn.tanh(tf.matmul(L1,W1)+b1)
#创建一个简单的神经网络
W2 = tf.Variable(tf.random_normal([100,10],0,0.02))
b2 = tf.Variable(tf.zeros([10]))
L2 = tf.nn.tanh(tf.matmul(L11,W2)+b2)
prediction = tf.nn.softmax(L2)
#二次代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
#使用梯度下降法
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss)  #学习率为0.001时准确率0.98
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)  #学习率为0.001时准确率0.9836

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(41):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Iter 0,Testing Accuracy 0.9216
Iter 1,Testing Accuracy 0.9491
Iter 2,Testing Accuracy 0.9604
Iter 3,Testing Accuracy 0.964
Iter 4,Testing Accuracy 0.9653
Iter 5,Testing Accuracy 0.9719
Iter 6,Testing Accuracy 0.9724
Iter 7,Testing Accuracy 0.9745
Iter 8,Testing Accuracy 0.9755
Iter 9,Testing Accuracy 0.9769
Iter 10,Testing Accuracy 0.9748
Iter 11,Testing Accuracy 0.9772
Iter 12,Testing Accuracy 0.9805
Iter 13,Testing Accuracy 0.9796
Iter 14,Testing Accuracy 0.98
Iter 15,Testing Accuracy 0.9765
Iter 16,Testing Accuracy 0.9804
Iter 17,Testing Accuracy 0.9801
Iter 18,Testing Accuracy 0.9809
Iter 19,Testing Accuracy 0.9814
Iter 20,Testing Accuracy 0.9806
Iter 21,Testing Accuracy 0.9801
Iter 22,Testing Accuracy 0.981
Iter 23,Testing Accuracy 0.9821
Iter 24,Testing Accuracy 0.9824
Iter 25,Testing Accuracy 0.9815
Iter 26,Testing Accuracy 0.9818
Iter 27,Testing Accuracy 0.9827
Iter 28,Testing Accuracy 0.9829
Iter 29,Testing Accuracy 0.9822
Iter 30,Testing Accuracy 0.981
Iter 31,Testing Accurac